[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/analiza-dokumentow/blob/main/16--cleaning.ipynb)

## Znaki specjalne, nadmiarowe spacje, znaczniki HTML, inne jeżeli potrzeba

## Czyszczenie danych
* istotne z perspektywy uwydatnienia poszukiwanych informacji



In [ ]:
test_text = '''Termin „post-truth” na tyle zyskał na znaczeniu w opisie rzeczywistości społeczno-politycznej, że w corocznym plebiscycie Oxford Dictionaries redaktorzy uznali go za słowo roku 2016. Argumentowano to m.in. tym, że termin „post-prawda”, używany coraz powszechniej w różnego rodzaju komentarzach politycznych i ważnych publikacjach, przestał mieć znaczenie peryferyjne, stając się terminem samodzielnym, niewymagającym dodatkowego wyjaśniania i definiowania.
'''

In [ ]:
test_text.split(".")

### Nietrywialny przykład z języka polskiego

* prof. Jan Kowalski obiecł zaliczenia w 1-szym terminie
* Koń ciągnie
* Nigdy więcej wojny!
* Ala ma kota a kot ma mleko

zdanie (ang. sentence) vs wypowiedź (ang. utterance)

In [ ]:
input_1 = "Ala ma kota a kot ma mleko. Krzyś ma psa, kota, chomika, jaszczurkę i papugę."
input_2 = "Wiedźmin rzucił się w jej stronę, w skoku dobywając miecza."
input_3 = "Stale ucz się. Im więcej różnych rzeczy wiesz, tym lepiej. Czytaj techniczną książkę raz na kwartał. Nietechniczne też czytaj. Uczestnicz w kursach, odwiedzaj konferencje."

## Tokenizacja

podzieleniu analizowanego tekstu na części ``tokeny``

``ala``, ``ma``, ``kota``, ``a``, ``kot``, ``ma``, ``mleko``

In [ ]:
#default - space
input_1.split()

https://www.nltk.org/

In [ ]:
pip install nltk

In [ ]:
import nltk


In [ ]:
from nltk.tokenize import word_tokenize

word_tokenize(input_2)

## Uni-Gram | Bi-Gram | Tri-Gram | N-Gram

Ala ma kota a kot ma mleko

```
Uni-Gram (1-gram) Ala, ma, kota, a, kot, ma, mleko
Bi-Gram (2-gram) Ala ma, ma kota, kota a, a kot, kot ma, ma mleko 
Tri-Gram (3-gram) Ala ma kota, ma kota a, kota a kot,  a kot ma, kot ma mleko 
...
```


In [ ]:
from nltk import ngrams
list(ngrams(word_tokenize(input_1), 3))

In [ ]:
from nltk import sent_tokenize
sent_tokenize(input_1)

In [ ]:
sent_tokenize('prof. Jan Kowalski obiecł zaliczenia w 1-szym terminie')

## Normalizacja

Upraszczanie do wspólnej wartości. 

```
kot -> kot
kota -> kot
kotem -> kot
jeden -> jeden
jedna -> jeden
jeden -> 1
```

## Stemming
bazując na definicji z angielskiej wikipedii jest to proces polegający na wydobyciu z wybranego wyrazu tzw. rdzenia, a więc tej jego części, która jest odporna na odmiany przez przyimki, rodzaje itp.

```
Wiedźmin -> Wiedźmin
rzucił -> rzucił
się -> się 
w -> w 
jej -> je
stronę -> stron
w -> w 
skoku -> skok
dobywając -> dobywa
miecza -> miecz
```

## Lematyzacja

pojęcie to jest bardzo podobne do powyższego, a oznacza sprowadzenie grupy wyrazów stanowiących odmianę danego zwrotu do wspólnej postaci, umożliwiającej traktowanie ich wszystkich jako te samo słowo.

```
Wiedźmin -> Wiedźmin
rzucił -> rzuca
się -> się 
w -> w 
jej -> jej
stronę -> strona
w -> w 
skoku -> skok
dobywając -> dobywa
miecza -> miecz
```

## Oczyszczanie | Przekształcanie 

In [ ]:
pip install pandas

In [ ]:
import pandas as pd

BOOKS = pd.read_pickle('var/books.pkl')

In [ ]:
BOOKS['content'][6][:255]

In [ ]:
BOOKS['content'][6][-1800:-1]

https://wolnelektury.pl/media/book/txt/dziady-dziady-widowisko-czesc-i.txt

```
-----
Ta lektura, podobnie jak tysiące innych, dostępna jest na stronie wolnelektury.pl.
Wersja lektury w opracowaniu merytorycznym i krytycznym (przypisy i motywy) dostępna jest na stronie http://wolnelektury.pl/katalog/lektura/dziady-dziady-widowisko-czesc-i.

Utwór opracowany został w ramach projektu Wolne Lektury przez fundację Nowoczesna Polska.
```

In [ ]:
test_text = '''
Adam Mickiewicz\r\n\r\nDziady.

ISBN 978-83-288-2972-5
Widowisko, część I\r\n\r\n\r\n\r\n/ Prawa strona teatru — Dziewica w samotnym pokoju — na boku ksiąg mnóstwo, fortepiano, okno z lewej strony w pole; na prawej wielkie zwierciadło; świeca gasnąca na stole i księga rozłożona (romans Vale
----- Ta lektura, podobnie jak tysiące innych, dostępna jest na stronie wolnelektury.pl.
Wersja lektury w opracowaniu merytorycznym i krytycznym (przypisy i motywy) dostępna jest na stronie http://wolnelektury.pl/katalog/lektura/dziady-dziady-widowisko-czesc-i.

Utwór opracowany został w ramach projektu Wolne Lektury przez fundację Nowoczesna Polska.
'''

In [ ]:
import re
def preprocess_text(text):
    text = re.sub(r"^.*\n","", text) 
    text = re.sub(r"ISBN.*[1-9\-]{16,20}","", text, flags=re.IGNORECASE) 
    
    text = re.sub(u"[ \n]+", " ", text) # newlines -> spaces
    text = re.sub(u"[ \r]+", " ", text) # \r -> spaces
    text = text.strip()
    text = re.sub(r"----- Ta lektura.*","", text)


    return text


assert ('\n' not in preprocess_text(test_text))
assert ('\r' not in preprocess_text(test_text))
assert ('  ' not in preprocess_text(test_text))
assert ('Ta lektura,' not in preprocess_text(test_text))
assert ('ISBN' not in preprocess_text(test_text))
assert ('2972-5' not in preprocess_text(test_text))


In [ ]:
test_text = preprocess_text(test_text)

In [ ]:
test_text

In [ ]:
def split_to_sentences(text):
  return [re.sub(r"^ ","",l) for l in re.split('\.|,|\?|!|:', text)]

assert(['hello world', "Hello John"] == split_to_sentences("hello world! Hello John"))

### interpunkcja

In [ ]:
import string 
string.punctuation

In [ ]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

In [ ]:
assert(remove_punct("hello, world!") == "hello world")

In [ ]:
remove_punct(test_text)

### Stop words

In [ ]:
pip install requests

In [ ]:
import requests

In [ ]:
stop_words = (requests
         .get('https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt')
         .text
         .split('\n'))

In [ ]:

stop_words[:10]

In [ ]:
def remove_stop_words(text):
  return " ".join([word for word in text.split(' ') if word not in stop_words])

assert "Cześć czołem" == remove_stop_words("Cześć i czołem")

In [ ]:
BOOKS.head()

In [ ]:
BOOKS['content_txt'] = (BOOKS.content
                             .apply(preprocess_text)
                             .apply(lambda s: s.lower())
                             .apply(remove_stop_words)
                             .apply(sent_tokenize)
                            )

In [ ]:
def clean_sentences(sentence):
    sentence = remove_punct(sentence)
    sentence.replace("—", "")
    
    return sentence

In [ ]:
BOOKS['sentences_clean'] = BOOKS.content_txt.apply(lambda sentences: list(map(clean_sentences, sentences)))

In [ ]:
BOOKS[:4]

In [ ]:
BOOK_LINES = BOOKS[['author',  'sentences_clean']].explode('sentences_clean')

In [ ]:
BOOK_LINES = BOOK_LINES.reset_index(drop=True)

In [ ]:
BOOK_LINES.head()

In [ ]:
BOOK_LINES.groupby('author').count()

Duża różnica w elementach danej klasy!! Może mieć wpływ na rezultaty

In [ ]:
BOOK_LINES['words'] = BOOK_LINES.sentences_clean.apply(lambda s: len(s.split()))

In [ ]:
BOOK_LINES.groupby('author')['words'].describe()


In [ ]:
BOOK_LINES = BOOK_LINES[BOOK_LINES['words'] != 0]

In [ ]:
BOOK_LINES.groupby('author')['words'].quantile(0.98)

In [ ]:
BOOK_LINES.groupby('author')['words'].describe()

In [ ]:
words_together = " ".join(BOOK_LINES[BOOK_LINES.author == 'Mickiewicz'].sentences_clean)

In [ ]:
BOOK_LINES.to_pickle('var/books_lines.pkl')

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(width = 400, height = 400, 
                      background_color = 'black', 
                      min_font_size = 5,
                      stop_words=stop_words,
                      max_words=1000,
                      collocations=False).generate("Ala ma kota a kot ma mleko")

plt.figure(figsize = (12, 12), facecolor = 'lavender')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad = 2) 
plt.show()
